In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>
#include <cmath>

std::cout << std::boolalpha;
// Use (void) to silent unused warnings.
#define assertm(exp, msg) assert(((void)msg, exp))

@0x7f79e83b1b60

In [2]:
#define VAR_NAME(Variable) (#Variable)

void print_tensor_size(std::string name, torch::Tensor t)
{
    int dims = t.dim();
    std::cout << name << " dims is (";
    for (int i = 0; i < dims; i++) {
        std::cout << t.size(i);
        if (i < (dims - 1)) std::cout << " x ";
    }
    std::cout << ")" << std::endl;
}

# 自定义层

构造一个没有任何参数的自定义层

In [3]:
/*design a net*/
struct CenteredLayer : torch::nn::Module {
  CenteredLayer() {
  }

  // Implement the Net's algorithm.
  torch::Tensor forward(torch::Tensor x) {    
    return x - x.mean();
  }
};

In [4]:
// instantiated the layer (as a NN network).
auto layer = std::make_shared<CenteredLayer>();

auto t = torch::tensor(at::ArrayRef<float>({1, 2, 3, 4, 5}));

In [5]:
printT(layer->forward(t));

layer->forward(t) = 
-2
-1
 0
 1
 2
[ CPUFloatType{5} ]
<<--->>



将层作为组件合并到构建更复杂的模型中

In [6]:
torch::nn::Sequential net(torch::nn::Linear(8, 128),
                          CenteredLayer()
                         );

auto X = torch::rand({4, 8});
printT(net->forward(X).mean());

net->forward(X).mean() = 
-3.72529e-09
[ CPUFloatType{} ]
<<--->>



带参数的图层

In [7]:
///In Python, we wrap the tensors with the torch.nn.Parameter class, while in C++ 
///we have to pass the tensor through the register_parameter method instead. 
///The reason for this is that the Python API can detect that an attribute is of type 
///torch.nn.Parameter and automatically registers such tensors.
///  https://pytorch.org/tutorials/advanced/cpp_frontend.html#using-the-pytorch-c-frontend

namespace F = torch::nn::functional;

struct MyLinear : torch::nn::Module {
  MyLinear(int64_t in_units, int64_t units) {
    weight = register_parameter("weight", torch::randn({in_units, units}));
    bias   = register_parameter("bias", torch::randn(units));
  }
  torch::Tensor forward(torch::Tensor X) {
    linear = torch::addmm(bias, X, weight);
    return F::relu(linear);
  }
  torch::Tensor weight, bias;
  torch::Tensor linear;
};

In [8]:
MyLinear dense(5,3);
printT(dense.weight);
printT(dense.weight.requires_grad());

dense.weight = 
 0.2861 -1.0340 -1.0330
-0.3161  0.0017 -0.1590
-0.8564  0.1962 -0.0161
-0.5919  0.7363 -1.3165
 0.5231 -0.3125 -0.7212
[ CPUFloatType{5,3} ]
<<--->>

dense.weight.requires_grad() = 
true
<<--->>



使用自定义层直接执行正向传播计算

In [9]:
printT(dense.forward(torch::rand({2, 5})));

dense.forward(torch::rand({2, 5})) = 
 0.0000  0.0000  0.8640
 0.0000  0.0000  0.0000
[ CPUFloatType{2,3} ]
<<--->>



使用自定义层构建模型

In [10]:
torch::nn::Sequential net2(MyLinear(64, 8), MyLinear(8, 1));
printT(net2->forward(torch::rand({2, 64})));

net2->forward(torch::rand({2, 64})) = 
 0
 0
[ CPUFloatType{2,1} ]
<<--->>

